## Import libraries

In [ ]:
import ee
import geemap

## Create an interactive map

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [ ]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      Chapter A3.9 Conservation Applications - Assessing the
#                spatial relationship between burned area and precipitation
#  Checkpoint:   A39b
#  Authors:      Harriet Branson, Chelsea Smith
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# ** Upload the area of interest ** #
AOI = ee.Geometry.Polygon([
[
[37.72, -11.22],
[38.49, -11.22],
[38.49, -12.29],
[37.72, -12.29]
]
])
Map.centerObject(AOI, 9)
Map.addLayer(AOI, {
    'color': 'white'
}, 'Area of interest')

# ** MODIS Monthly Burn Area ** #

# Load in the MODIS Monthly Burned Area dataset.
dataset = ee.ImageCollection('MODIS/006/MCD64A1') \
.filter(ee.Filter.date('2010-01-01', '2021-12-31'))

# Select the BurnDate band from the images in the collection.
MODIS_BurnDate = dataset.select('BurnDate')

# A function that will calculate the area of pixels in each image by date.

def func_rlx(img):
    area = ee.Image.pixelArea() \
    .updateMask(
    img
    ) \
    .divide(1e6) \
    .clip(AOI) \
    .reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = AOI,
        scale = 500,
        bestEffort = True
        ).getNumber(
    'area'
    ); 
    # Add a new band to each image in the collection named area.
    return img.addBands(ee.Image(area).rename('area'))

addArea = func_rlx


















# Apply function on image collection.
burnDateArea = MODIS_BurnDate.map(addArea)

# Select only the area band as we are using system time for date.
burnedArea = burnDateArea.select('area')

# Create a chart that shows the total burned area over time.
burnedAreaChart = \
ui.Chart.image \
.series(
imageCollection = burnedArea, 
region = AOI,
reducer = ee.Reducer.mean(),
scale = 500,
xProperty = 'system =time_start' 
) \
.setSeriesNames(['Area']) \
.setOptions(
title = 'Total monthly area burned in AOI',
hAxis = {
    'title': 'Date', 
    format: 'YYYY', 
    'gridlines': {
            'count': 12
        },
    titleTextStyle = {
            'italic': False,
            'bold': True
        }
},
vAxis = {
    'title': 'Total burned area (km²)', 
    'maxValue': 2250, 
    'minValue': 0,
    'titleTextStyle': {
            'italic': False,
            'bold': True
        }
},
lineWidth = 1.5,
colors = ['d74b46'], 
)
print(burnedAreaChart.getInfo())

# -----------------------------------------------------------------------
# CHECKPOINT
# -----------------------------------------------------------------------

# Load in the CHIRPS rainfall pentad dataset.
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

# Define the temporal range
startyear = 2010
endyear = 2021

# Set the advancing dates from the temporal range.
startdate = ee.Date.fromYMD(startyear, 1, 1)
enddate = ee.Date.fromYMD(endyear, 12, 31)

# Create a list of years
years = ee.List.sequence(startyear, endyear)
# Create a list of months
months = ee.List.sequence(1, 12)

# Filter the dataset based on the temporal range.
Pchirps = chirps.filterDate(startdate, enddate) \
.sort('system:time_start',
False) \
.filterBounds(AOI) \
.select('precipitation'); 

# Calculate the precipitation per month.
MonthlyRainfall = ee.ImageCollection.fromImages(
y
) { 

    def func_dfl(m):
            w = Pchirps.filter(ee.Filter \
            .calendarRange(y, y, 'year')) \
            .filter(ee.Filter.calendarRange(m, m,
            'month')) \
            .sum(); 
            return w.set('year', y) \
            .set('month', m) \
            .set('system:time_start', ee.Date \
            .fromYMD(y, m, 1).millis()
            ) \
            .set('date', ee.Date.fromYMD(y, m,
            1))

    return months.map(func_dfl)













}).flatten())
# Print the image collection.
print('Monthly Precipitation Image Collection', MonthlyRainfall.getInfo())

# ** Chart: CHIRPS Precipitation ** #

# Create a chart displaying monthly rainfall over a temporal range.
monthlyRainfallChart = \
ui.Chart.image \
.series(
imageCollection = MonthlyRainfall.select(
'precipitation'), 
region = AOI,
reducer = ee.Reducer \
.mean(), 
scale = 500,
xProperty = 'system =time_start' 
) \
.setSeriesNames(['Precipitation']) \
.setOptions(
title = 'Total monthly precipitation in AOI', 
hAxis = {
    'title': 'Date',
    format: 'YYYY', 
    'gridlines': {
            'count': 12
        },
    titleTextStyle = {
            'italic': False,
            'bold': True
        }
},
vAxis = {
    'title': 'Precipitation (mm)', 
    'maxValue': 450, 
    'minValue': 0,
    'titleTextStyle': {
            'italic': False,
            'bold': True
        }
},
lineWidth = 1.5,
colors = ['4f5ebd'],
)
print(monthlyRainfallChart.getInfo())

# 2010/2011 wet season total
year = 2010; 
startDate = ee.Date.fromYMD(year, 11, 1); 
endDate = ee.Date.fromYMD(year + 1, 5, 31); 
filtered = chirps \
.filter(ee.Filter.date(startDate, endDate))
Rains10_11Total = filtered.reduce(ee.Reducer.sum()).clip(AOI)

# 2011/2012 wet season total
year = 2011; 
startDate = ee.Date.fromYMD(year, 11, 1); 
endDate = ee.Date.fromYMD(year + 1, 5, 31); 
filtered = chirps \
.filter(ee.Filter.date(startDate, endDate))
Rains11_12Total = filtered.reduce(ee.Reducer.sum()).clip(AOI)

# -----------------------------------------------------------------------
# CHECKPOINT
# -----------------------------------------------------------------------

## Display the interactive map

In [ ]:
Map